In [64]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm


In [65]:
tomtom = pd.read_parquet("tomtom_data.parquet")
vessels = pd.read_parquet("vessels_data.parquet")
sensors_location = pd.read_csv("sensor-location.xlsx - Sheet1.csv", )
sensors = pd.read_csv("sensordata_SAIL2025.csv", parse_dates=["timestamp"])

sensors_location["Effectieve breedte"] = (
    sensors_location["Effectieve breedte"]
    .astype(str)
    .str.replace(",", ".")
    .astype(float)
)

sensors_location[["lat", "lon"]] = (
    sensors_location["Lat/Long"]
    .str.replace(" ", "")   # remove spaces
    .str.split(",", expand=True)
    .astype(float)
)

sensors_location = sensors_location.rename(columns={"Objectummer": "sensor_id"})

In [66]:
# vessels.info()
# vessels.head()
# sensors.info
# sensors.head()
# sensors_location.head()
# sensors_location.info()




In [67]:
#Find Flow per sensor per meter
#Divide by 3 minutes and effective width

width = sensors_location.set_index("sensor_id")["Effectieve breedte"].to_dict()

# Divide each matching visitor column by its sensor’s width
for col in sensors.columns:
    if "_" in col:  # e.g. CMSA-GAKH-01_0
        sensor_id = col.split("_")[0]
        if sensor_id in width:
            sensors[col] = sensors[col] / (3*width[sensor_id])


In [68]:
#Combine vessel data into 3 min bins, taking the average position of the vessel.
vessels["timestamp"] = pd.to_datetime(vessels["timestamp"], utc=True, errors="coerce")
vessels["timestamp"] = vessels["timestamp"].dt.floor("3min")

vessels = (
    vessels.groupby(["timestamp", "imo-number"], as_index=False)
    .agg({
        "lat": "mean",
        "lon": "mean",
        "length": "first"
    })
)

In [69]:
#Combining vessel and sensor data.
vessels["timestamp"] = pd.to_datetime(vessels["timestamp"], utc=True)
sensors["timestamp"] = pd.to_datetime(sensors["timestamp"], utc=True)

combined = sensors.merge(vessels, on="timestamp", how="inner")

combined.head(100)


,timestamp,CMSA-GAKH-01_0,CMSA-GAKH-01_180,CMSA-GAWW-11_120,CMSA-GAWW-11_300,CMSA-GAWW-12_115,CMSA-GAWW-12_295,CMSA-GAWW-13_120,CMSA-GAWW-13_300,CMSA-GAWW-14_40,...,hour,minute,day,month,weekday,is_weekend,imo-number,lat,lon,length
0,2025-08-20 06:27:00+00:00,0.000000,0.099502,0.392157,0.588235,0.641026,0.25641,0.151515,0.000000,0.185185,...,8,27,20,8,2,0,4becccd38142efcac823c94e99e4c9f918dc47de027d83...,52.059721,3.616344,14000.0
1,2025-08-20 06:27:00+00:00,0.000000,0.099502,0.392157,0.588235,0.641026,0.25641,0.151515,0.000000,0.185185,...,8,27,20,8,2,0,64a15227c0878ef130cc4e6c27fdc668b69e528a557b63...,53.661851,5.537259,19900.0
2,2025-08-20 06:27:00+00:00,0.000000,0.099502,0.392157,0.588235,0.641026,0.25641,0.151515,0.000000,0.185185,...,8,27,20,8,2,0,a867f56d1cd3b9b6812798549281aa82c60f5cebcf5f61...,52.086418,3.423325,19900.0
3,2025-08-20 06:27:00+00:00,0.000000,0.099502,0.392157,0.588235,0.641026,0.25641,0.151515,0.000000,0.185185,...,8,27,20,8,2,0,b2eb0407183a1fabe106857e32added9ec78ac9c079149...,51.947955,4.052420,33600.0
4,2025-08-20 06:30:00+00:00,0.199005,0.149254,0.588235,0.392157,0.769231,0.25641,0.151515,0.151515,0.555556,...,8,30,20,8,2,0,2d18f769b33844885ab5d48e9b4c0764308170355a4245...,53.410521,4.589400,8900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2025-08-20 06:33:00+00:00,0.149254,0.199005,0.784314,0.490196,0.000000,0.25641,0.000000,0.151515,0.648148,...,8,33,20,8,2,0,508607cb85db6e98b5293f54b5002389f6e18c082303f0...,52.406696,4.816781,5900.0
96,2025-08-20 06:33:00+00:00,0.149254,0.199005,0.784314,0.490196,0.000000,0.25641,0.000000,0.151515,0.648148,...,8,33,20,8,2,0,51642004ecb740656103b2b0c3d4ce286db2253a1d930d...,52.406776,4.882029,8800.0
97,2025-08-20 06:33:00+00:00,0.149254,0.199005,0.784314,0.490196,0.000000,0.25641,0.000000,0.151515,0.648148,...,8,33,20,8,2,0,51de12cb55ef030d7e33cd76f8c5badcae3330621021a1...,52.100666,4.267667,5100.0
98,2025-08-20 06:33:00+00:00,0.149254,0.199005,0.784314,0.490196,0.000000,0.25641,0.000000,0.151515,0.648148,...,8,33,20,8,2,0,52bddda6694714952ff0993167c638d56677f44f639d87...,52.405651,4.818190,4100.0


In [70]:
# Merge sensor coordinates into the combined dataframe
lat_lookup = sensors_location.set_index("sensor_id")["lat"].to_dict()
lon_lookup = sensors_location.set_index("sensor_id")["lon"].to_dict()



In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius (meters)
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

# --- Compute distance for every vessel at every timestamp to every sensor ---
for _, sensor in sensors_location.iterrows():
    sensor_id = sensor["sensor_id"]
    s_lat = sensor["lat"]
    s_lon = sensor["lon"]
    
    # Distance is computed for *each vessel row* — time is automatically respected
    combined[f"dist_{sensor_id}"] = haversine(s_lat, s_lon, combined["lat"], combined["lon"])
    combined= combined[f"dist_{sensor_id}"].where(combined[f"dist_{sensor_id}"] < 1000, np.inf)

print(combined.head())


                  timestamp  CMSA-GAKH-01_0  CMSA-GAKH-01_180  \
0 2025-08-20 06:27:00+00:00        0.000000          0.099502   
1 2025-08-20 06:27:00+00:00        0.000000          0.099502   
2 2025-08-20 06:27:00+00:00        0.000000          0.099502   
3 2025-08-20 06:27:00+00:00        0.000000          0.099502   
4 2025-08-20 06:30:00+00:00        0.199005          0.149254   

   CMSA-GAWW-11_120  CMSA-GAWW-11_300  CMSA-GAWW-12_115  CMSA-GAWW-12_295  \
0          0.392157          0.588235          0.641026           0.25641   
1          0.392157          0.588235          0.641026           0.25641   
2          0.392157          0.588235          0.641026           0.25641   
3          0.392157          0.588235          0.641026           0.25641   
4          0.588235          0.392157          0.769231           0.25641   

   CMSA-GAWW-13_120  CMSA-GAWW-13_300  CMSA-GAWW-14_40  ...  dist_GASA-01-A1  \
0          0.151515          0.000000         0.185185  ...       